In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.python.ops import summary_ops_v2
from tensorflow.python.keras.callbacks import keras_model_summary
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPool2D, Input, Dense, Flatten, Dropout, Concatenate, Layer, LeakyReLU, Reshape, AveragePooling2D, Add

import numpy as np

import matplotlib.pyplot as plt

import cv2

import os
import datetime
from time import perf_counter, sleep
import threading
import traceback

from random import sample

from functools import partial

In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [ ]:
from progressive_gan import ProgressiveGAN
from utils import ImageGenerator, TensorBoardCallback, FitCheckpoint

In [ ]:
img_gen = ImageGenerator(r'E:\Workspace\datasets\cats\train',
                         batch_size=128, image_channels=3, fade=True,
                         preload_images=True, preload_images_size=128) # r'E:\Workspace\datasets\b\train_1\512'E:\Workspace\datasets\cats\train'E:\Workspace\datasets\mnist_imgs'

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=1e-5, beta_1=0., beta_2=.99, epsilon=1e-8)
# optimizer = keras.optimizers.RMSprop(learning_rate=5e-5)

progan = ProgressiveGAN(
    latent_dim=512,
    initial_image_size=4,
    final_image_size=128,
    image_channels=3,
    discriminator_optimizer=optimizer,
    gan_optimizer=optimizer)

tensorboard_callback = TensorBoardCallback('./logs', progan, image_generator_preview_save_interval=500, frame_generator_preview_save_interval=50, use_tensorboard=False)

progan.fit(img_gen, batches_per_step=100000, discriminator_train_per_gan_train=1, checkpoint_save_interval=10000, tensorboard_callback=tensorboard_callback)

In [ ]:
checkpoint_idx = 1

optimizer = keras.optimizers.RMSprop(learning_rate=2e-5)

loaded_progan = ProgressiveGAN.load(
    f'./model/model_checkpoint_{checkpoint_idx}/',
    discriminator_optimizer=optimizer,
    gan_optimizer=optimizer)

loaded_checkpoint = FitCheckpoint.load(f'./fit_checkpoints/checkpoint_{checkpoint_idx}/')

tensorboard_callback = TensorBoardCallback('./logs', loaded_progan, image_generator_preview_save_interval=500, frame_generator_preview_save_interval=50, use_tensorboard=False)

loaded_progan.resume_fit_from_checkpoint(loaded_checkpoint, image_generator=img_gen, tensorboard_callback=tensorboard_callback)

In [ ]:
# batch = np.concatenate([img_gen.get_batch(), img_gen.get_batch()], axis=0)
# print(batch.shape)
# print(batch.min())
# print(batch.max())

# plt.figure(figsize=(16, 8))

# plt.imshow(np.vstack([np.hstack([batch[i + 8*j] for i in range(8)]) for j in range(4)])/2 + .5)

# plt.show()

In [ ]:

latent_noise = progan.sample_latent_space(8)

generated_images = np.zeros((8*len(progan.generator), 32, 32, 3))
for i in range(len(progan.generator)):
    g = progan.generator[i][0].predict(latent_noise)
    for j in range(8):
        img = g[j,]
        img = (img + 1.)/2.
        generated_images[8*i + j,] = cv2.resize(img, (32, 32), interpolation=cv2.INTER_NEAREST)
    
print(generated_images.shape)
print(generated_images.min())
print(generated_images.max())

plt.figure(figsize=(16, 2*len(progan.generator)))

plt.imshow(np.vstack([np.hstack([generated_images[i + 8*j] for i in range(8)]) for j in range(len(progan.generator))])/2 + .5, interpolation=None)

plt.show()